In [83]:
import json
import pandas as pd
import os

In [84]:
path1 = "nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/infer-out_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/report.json"
path2 = "nacos_6f5b03d73ed64b035e9300de93465f53f77085f3/infer-out_6f5b03d73ed64b035e9300de93465f53f77085f3/report.json"

json1 = {}
json2 = {}

with open(path1, 'r') as f:
    json1 = json.load(f)

with open(path2, 'r') as f:
    json2 = json.load(f)

print(json1)
print(json2)

[{'bug_type': 'NULL_DEREFERENCE', 'qualifier': 'object returned by `getProperty("os.name")` could be null and is dereferenced at line 45.', 'severity': 'ERROR', 'line': 45, 'column': -1, 'procedure': 'com.alibaba.nacos.common.pathencoder.PathEncoderManager.<init>()', 'procedure_start_line': 41, 'file': 'common/src/main/java/com/alibaba/nacos/common/pathencoder/PathEncoderManager.java', 'bug_trace': [{'level': 0, 'filename': 'common/src/main/java/com/alibaba/nacos/common/pathencoder/PathEncoderManager.java', 'line_number': 41, 'column_number': -1, 'description': 'start of procedure PathEncoderManager()'}, {'level': 0, 'filename': 'common/src/main/java/com/alibaba/nacos/common/pathencoder/PathEncoderManager.java', 'line_number': 39, 'column_number': -1, 'description': ''}, {'level': 0, 'filename': 'common/src/main/java/com/alibaba/nacos/common/pathencoder/PathEncoderManager.java', 'line_number': 43, 'column_number': -1, 'description': ''}, {'level': 1, 'filename': 'common/src/main/java/c

In [85]:
# create pandas dataframe from json
df1 = pd.json_normalize(json1)
df2 = pd.json_normalize(json2)

# create a table from dataframes indicating the number of bug types retrieved from bug_type column
df1_bug_type = df1['bug_type'].value_counts()
df2_bug_type = df2['bug_type'].value_counts()
print("Number of bugs in first JSON: ", len(df1))
print("Number of bugs in second JSON: ", len(df2))
print("Number of bugs in first JSON per bug type: ", df1_bug_type)
print("Number of bugs in second JSON per bug type: ", df2_bug_type)

Number of bugs in first JSON:  208
Number of bugs in second JSON:  216
Number of bugs in first JSON per bug type:  bug_type
THREAD_SAFETY_VIOLATION        139
NULL_DEREFERENCE                53
RESOURCE_LEAK                    9
INEFFICIENT_KEYSET_ITERATOR      5
INTERFACE_NOT_THREAD_SAFE        2
Name: count, dtype: int64
Number of bugs in second JSON per bug type:  bug_type
THREAD_SAFETY_VIOLATION        144
NULL_DEREFERENCE                58
RESOURCE_LEAK                    7
INEFFICIENT_KEYSET_ITERATOR      6
INTERFACE_NOT_THREAD_SAFE        1
Name: count, dtype: int64


In [86]:
# Discard the rows with bug type = "INEFFICIENT_KEYSET_ITERATOR", "INTERFACE_NOT_THREAD_SAFE", "THREAD_SAFETY_VIOLATION"
df1 = df1[df1.bug_type != "INEFFICIENT_KEYSET_ITERATOR"]
df1 = df1[df1.bug_type != "INTERFACE_NOT_THREAD_SAFE"]
df1 = df1[df1.bug_type != "THREAD_SAFETY_VIOLATION"]
df2 = df2[df2.bug_type != "INEFFICIENT_KEYSET_ITERATOR"]
df2 = df2[df2.bug_type != "INTERFACE_NOT_THREAD_SAFE"]
df2 = df2[df2.bug_type != "THREAD_SAFETY_VIOLATION"]

In [87]:
# remove the rows with the same hash and node_key pairs within df1 (for example if there are 3 duplicates, remove two of them; one should remain)
df1 = df1.drop_duplicates(subset=['hash', 'node_key'], keep='first')
df2 = df2.drop_duplicates(subset=['hash', 'node_key'], keep='first')

In [88]:
# We need to compare the columns hash and node_key between two dataframes. Discard the rows with different hash and node_key
df1_neverfixed = df1[df1.hash.isin(df2.hash) & df1.node_key.isin(df2.node_key)]
print(df1_neverfixed.shape)
print(df1_neverfixed)

df1_fixed = df1[~df1.hash.isin(df2.hash) & ~df1.node_key.isin(df2.node_key)]
print(df1_fixed.shape)
print(df1_fixed)


(28, 14)
            bug_type                                          qualifier  \
3   NULL_DEREFERENCE  object returned by `getPageItems(page)` could ...   
4   NULL_DEREFERENCE  object returned by `getPageItems(page)` could ...   
10  NULL_DEREFERENCE  object `clientFactory` last assigned on line 7...   
12  NULL_DEREFERENCE  object `clientFactory` last assigned on line 8...   
15  NULL_DEREFERENCE  object returned by `getPageItems(page)` could ...   
16  NULL_DEREFERENCE  object returned by `getPageItems(page)` could ...   
23  NULL_DEREFERENCE  object `ex` last assigned on line 139 could be...   
26  NULL_DEREFERENCE  object `file` last assigned on line 150 could ...   
29  NULL_DEREFERENCE  object returned by `sourceService.getJdbcTempl...   
31  NULL_DEREFERENCE  object returned by `getMetricsKeys(clientMetri...   
33     RESOURCE_LEAK  resource of type `java.io.FileInputStream` acq...   
37  NULL_DEREFERENCE  object `file` last assigned on line 220 could ...   
40     RESOURCE_

In [89]:
# We need to compare the columns hash and node_key between two dataframes. Discard the rows with different hash and node_key
df2_neverfixed = df2[df2.hash.isin(df1.hash) & df2.node_key.isin(df1.node_key)]
print(df2_neverfixed.shape)
print(df2_neverfixed)

df2_fixed = df2[~df2.hash.isin(df1.hash) & ~df2.node_key.isin(df1.node_key)]
print(df2_fixed.shape)
print(df2_fixed)

(28, 14)
            bug_type                                          qualifier  \
5   NULL_DEREFERENCE  object returned by `getPageItems(page)` could ...   
7   NULL_DEREFERENCE  object returned by `getPageItems(page)` could ...   
9   NULL_DEREFERENCE  object `clientFactory` last assigned on line 7...   
13  NULL_DEREFERENCE  object `clientFactory` last assigned on line 8...   
14  NULL_DEREFERENCE  object returned by `getPageItems(page)` could ...   
15  NULL_DEREFERENCE  object returned by `getPageItems(page)` could ...   
20  NULL_DEREFERENCE  object returned by `getMetricsKeys(clientMetri...   
21  NULL_DEREFERENCE  object `ex` last assigned on line 137 could be...   
26  NULL_DEREFERENCE  object `file` last assigned on line 150 could ...   
27  NULL_DEREFERENCE  object returned by `sourceService.getJdbcTempl...   
31     RESOURCE_LEAK  resource of type `java.io.FileInputStream` acq...   
33  NULL_DEREFERENCE  object `file` last assigned on line 220 could ...   
34     RESOURCE_

In [90]:
# create a new directory called datatset if there is no such directory
if not os.path.exists("dataset"):
    os.mkdir("dataset")

In [91]:
procedure_string = "// >>>>>>>>>>>>>>>> Procedure Start: <<<<<<<<<<<<<<<<"
bug_string = "// >>>>>>>>>>>>>>>>>>>>>> BUG: <<<<<<<<<<<<<<<<<<<<<<"

for index, row in df1_neverfixed.iterrows():
    # Construct the full path for each file
    path = "nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/" + row['file']
    print(path)

    content = ""

    with open(path, 'r') as file:
        content = file.read()

    content = content.split("\n")
    content.insert(row["procedure_start_line"] - 1, procedure_string)
    content.insert(row["line"], bug_string)

    content = "\n".join(content)

    file_name = row['file'].split("/")[-1]
    
    file_name_orig = file_name
    # if file exists
    count = 0
    while(os.path.exists("dataset/" + file_name)):
        count += 1
        file_name = file_name_orig.split(".")[0] + str(count) + "." + file_name_orig.split(".")[1]

    with open("dataset/" + file_name, 'w') as file:
        file.write(content)

nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/persistence/src/main/java/com/alibaba/nacos/persistence/repository/embedded/EmbeddedPaginationHelperImpl.java
nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/persistence/src/main/java/com/alibaba/nacos/persistence/repository/extrnal/ExternalStoragePaginationHelperImpl.java
nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/naming/src/main/java/com/alibaba/nacos/naming/core/v2/client/manager/impl/ConnectionBasedClientManager.java
nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/naming/src/main/java/com/alibaba/nacos/naming/core/v2/client/manager/impl/ConnectionBasedClientManager.java
nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/persistence/src/main/java/com/alibaba/nacos/persistence/repository/embedded/EmbeddedPaginationHelperImpl.java
nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/persistence/src/main/java/com/alibaba/nacos/persistence/repository/extrnal/ExternalStoragePaginationHelperImpl.java
nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/cor

nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/istio/target/generated-sources/protobuf/java/istio/networking/v1alpha3/DestinationRuleOuterClass.java
nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/istio/target/generated-sources/protobuf/java/istio/networking/v1alpha3/DestinationRuleOuterClass.java
nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/istio/target/generated-sources/protobuf/java/istio/networking/v1alpha3/DestinationRuleOuterClass.java
nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/istio/target/generated-sources/protobuf/java/istio/networking/v1alpha3/EnvoyFilterOuterClass.java
nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/istio/target/generated-sources/protobuf/java/istio/networking/v1alpha3/EnvoyFilterOuterClass.java
nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/istio/target/generated-sources/protobuf/java/istio/networking/v1alpha3/VirtualServiceOuterClass.java
nacos_2bb93982dfa89e0c9c9fa1bd31a44f983c00f27d/istio/target/generated-sources/protobuf/java/istio/networking/v1alph

In [92]:
if not os.path.exists("reports"):
    os.mkdir("reports")

with open ("reports/neverfixed_bug_report.json", 'w') as file:
    file.write(df1_neverfixed.to_json(orient='records'))

with open ("reports/neverfixed_bug_report.csv", 'w') as file:
    file.write(df1_neverfixed.to_csv(index=False))

df1_neverfixed.to_excel("reports/neverfixed_bug_report.xlsx", index=False)

with open ("reports/fixed_bug_report.json", 'w') as file:
    file.write(df1_fixed.to_json(orient='records'))

with open ("reports/fixed_bug_report.csv", 'w') as file:
    file.write(df1_fixed.to_csv(index=False))

df1_fixed.to_excel("reports/fixed_bug_report.xlsx", index=False)